Here, we will get the festival happening in Switzerland, using data from:
[BandsInTown API](http://www.bandsintown.com/api/overview).

## 1. Scrapping
Data that will be received and stored:
- 'id'
- 'bandsintown url'
- 'datetime'
- 'artists'
    - per artist:
        - 'name'
        - 'bandsintown url'
- 'venue'
    - 'id'
    - 'name'
    - 'bandsintown url'
    - 'city'
    - 'region'
    - 'latitude'
    - 'longitude'

Limitations
* The API limits the number of returned results, so a request for every day will be done.
* The API limits the radius to 150 miles so several cities will be specified. To avoid overlap, the id of the events will be checked
Cities : Geneva, Lausanne, Zurich, Berne, Basel, Lugano, Lucerne



In [1]:
import requests 
import pandas as pd
from scipy import stats
import urllib3
import json
% matplotlib inline
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt

In [2]:
base_url = 'http://api.bandsintown.com/events/search.json?api_version=2.0&app_id=epfl&radius=50'
cities = ['Geneva,Switzerland',
         'Lausanne,Switzerland',
         'Lucerne,Switzerland'
         'Zurich,Switzerland',
         'Bern,Switzerland',
         'Basel,Switzerland',
         'Locarno,Switzerland',
         ]
starting_year =  2016
ending_year =  2017


We will now loop over

In [ ]:
def bandsInTownRequest(date_request,city_request):
    request_url = base_url + '&location=' + city_request + '&date=' + date_request
    data_json = requests.get(request_url).json()
    return data_json
    
def fillPandasJson(data_json):
        df_city = pd.io.json.json_normalize(data_json,meta_prefix='event_',
                                            meta=['id','datetime','url',
                                                 ['venue','name'],
                                                 ['venue','url'],
                                                 ['venue','id'],
                                                 ['venue','city'],
                                                 ['venue','region'],
                                                 ['venue','country'],
                                                 ['venue','latitude'],
                                                 ['venue','longitude']]
                                            ,record_path='artists',record_prefix='artist_')
        if len(df_city)>0:
            df_city = df_city[df_city['event_venue.country'] != 'France']
            df_city.drop_duplicates(inplace = True)
            df_city.drop(['artist_mbid','event_venue.country'],axis = 1, inplace=True)
        return df_city


In [ ]:
#Init
df = pd.DataFrame()

for year in range(starting_year,ending_year):
    for month in range (1,12):
        print('Now processing Month/Year:',month,'/',year)
        for day in range (1,31):
            date = str(year) + '-' + str("%02d" % month) + '-' + str("%02d" % day)
            date_request = date+'-'+date
            for city_request in cities:
                data_json = bandsInTownRequest(date_request,city_request)
                df_city = fillPandasJson(data_json)
                df = pd.concat([df,df_city])

Now processing Month/Year: 1 / 2016


In [ ]:
df